Import Dependencies

In [337]:
import urllib
from bs4 import BeautifulSoup as bs
import nltk
import pandas  as pd

In [362]:
df = pd.read_csv('news_summary.csv')
i = 2

Extract data from html files

In [363]:
url = df['read_more'][i]
gold_summary = df['text'][i]
if "indiatoday" in url or "intoday" in url:
    baseurl = "i"
elif "hindustantimes" in url:
    baseurl = "h"
elif "theguardian" in url:
    baseurl = "t"
file = urllib.urlopen(url)
html = file.read()

<h2>Parse the data to extract article text</h2>

In [364]:
soup = bs(html, 'html.parser')

if baseurl == "i":
    entries = soup.find_all('div',{'class':'description'})
elif baseurl == "t":
    entries = soup.find_all('div',{'itemprop':'articleBody'})
elif baseurl == "h":
    entries = soup.find_all('div',{'itemprop':'articlebody'})

for each in entries:
        if each.figure:
            each.figure.decompose()
content = []
for e in entries:
     content.extend(e.find_all("p"))

text = ""
for each in content:
    text = text + each.get_text() +" "
    
text = text.encode('utf-8').decode("unicode_escape").encode('ascii','ignore')
print text
text = nltk.sent_tokenize(text)    
    
# for each in text:
#     print each , "\n"

The Indira Gandhi Institute of Medical Sciences (IGIMS) in Patna amended its marital declaration form on Thursday, replacing the word virgin with unmarried after controversy. Until now, new recruits to the super-specialty medical institute in the state capital were required to declare if they were bachelors, widowers or virgins. IGIMS medical superintendent Dr Manish Mandal said institute director Dr NR Biswas held a meeting on Thursday morning before directing that the word virgin on the marital declaration form be immediately replaced with unmarried. Dr Biswas had just returned after a four-day leave of absence. Earlier, Bihar health minister Mangal Pandey had ended up redefining the very meaning of virginity in his attempts to justify the awkward phrasing of the question in the form. Following a public furore over the document on Wednesday, the minister told news channels that there was nothing wrong with using the word virgin because it simply meant kanya or kunwari  which means an

Start extraction

In [365]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter

In [366]:
# tokenize the article
tokenizer = RegexpTokenizer(r'\w+')
word_list=[]
for t in text:
    word_list.extend(tokenizer.tokenize(t))


# remove stop words from the article
filtered_words = [word for word in word_list if word.lower() not in stopwords.words('english')]
total_words = len(filtered_words)
words = filtered_words

# find n-gram probability for filtered words
filtered_words  = []
for each in Counter(words).items():
    filtered_words.append([each[0] , float(each[1])/float(total_words)])

# identify important words
important_words = []
for item in filtered_words:
    if item[1]>0.003:
        important_words.append(item[0])

while '. ' in text:
    text = text.replace('. ','\n') 
# calculate sentence score
while "\n" in text:
    text = text.split("\n")

print text


# sentenceScore  = []
# for sentence in text:
#     score = 0
#     count = 0
#     for word in important_words:
#         if word not in stopwords.words('english'):
#             count = count +1
#         if word in sentence and word not in stopwords.words('english'):
#             score = score + 1
#             count = count + 1
#     sentenceScore.append([sentence,float(score)/float(len(sentence))**(1/2)])

# # extract top n/5 sentences (n is total number of sentences in the text)
# sen = sentenceScore
# sentenceScore.sort(key=lambda x: x[1], reverse=True)
# print len(sentenceScore)
# cutScore = sentenceScore[len(sentenceScore)/5][1]
# print cutScore
# count = 0
# machine_summary = ""

# for sentence in sen:
#     if sentence[1]>cutScore:
#         print sentence[0] + "\n"
#         machine_summary = machine_summary + sentence[0] + " "


# print machine_summary

['The Indira Gandhi Institute of Medical Sciences (IGIMS) in Patna amended its marital declaration form on Thursday, replacing the word virgin with unmarried after controversy.', 'Until now, new recruits to the super-specialty medical institute in the state capital were required to declare if they were bachelors, widowers or virgins.', 'IGIMS medical superintendent Dr Manish Mandal said institute director Dr NR Biswas held a meeting on Thursday morning before directing that the word virgin on the marital declaration form be immediately replaced with unmarried.', 'Dr Biswas had just returned after a four-day leave of absence.', 'Earlier, Bihar health minister Mangal Pandey had ended up redefining the very meaning of virginity in his attempts to justify the awkward phrasing of the question in the form.', 'Following a public furore over the document on Wednesday, the minister told news channels that there was nothing wrong with using the word virgin because it simply meant kanya or kunwar

In [367]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [368]:
vect = TfidfVectorizer(stop_words='english')
dtm = vect.fit_transform(text)
features = vect.get_feature_names()

In [369]:
print dtm.toarray()

[[ 0.          0.          0.         ...,  0.          0.20086869  0.        ]
 [ 0.          0.          0.         ...,  0.29966644  0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.15673749  0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.34546787  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [370]:
scores = zip(vect.get_feature_names(),
                 np.asarray(dtm.sum(axis=0)).ravel())
sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
for item in sorted_scores:
    print "{0:50} Score: {1}".format(item[0], item[1])

form                                               Score: 1.20307416209
institute                                          Score: 0.957979323844
dr                                                 Score: 0.873207740823
declaration                                        Score: 0.871956740827
marital                                            Score: 0.871956740827
virgin                                             Score: 0.868017020437
word                                               Score: 0.868017020437
asked                                              Score: 0.740885786724
medical                                            Score: 0.728546379931
just                                               Score: 0.719802601636
question                                           Score: 0.693354734939
virginity                                          Score: 0.660159839907
said                                               Score: 0.635248655106
pandey                                             S

In [371]:
# import numpy as np

# #review_id = np.random.randint(0, len(text))
# review_text = words
# review_length = len(words)

# # create a dictionary of words and their TF-IDF scores
# word_scores = {}
# for word in words:
# #     word = word.lower()
#     if word in features:
# #         print word
# #         print dtm[review_id, features.index(word)]
#          #word_scores[word] = dtm[review_id, features.index(word)]
#          word_scores[word] = dtm[10, features.index(word)]

# print word_scores
# # print words with the top 5 TF-IDF scores
# print 'TOP SCORING WORDS:'
# top_scores = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)[:10]
# for word, score in top_scores:
#     print word

In [372]:
sorted_scores = dict(sorted_scores)


In [373]:
sentence_score = []
for each in text:
    score = 0.0
#     each = each.
    for word in each.split(" "):
        #print word
        word = word.lower()
        if word in sorted_scores:
#             print "Hello"
            score += sorted_scores[word]
#             print word_scores[word]
    sentence_score.append([each,score])    
        

In [374]:
machine_summary = ""
sentences = []
for each in sentence_score:
    if each[1] != 0.0:
        machine_summary = machine_summary + each[0]
        sentences.append([each[0],each[1]])
        

In [381]:
top_sentences = sorted(dict(sentences).items(), key=lambda x: x[1], reverse=True)[:2]
machine_summary = ""
for each in top_sentences:
    print each
    machine_summary = machine_summary + each[0]
    #print top_sentences[each],each

('IGIMS medical superintendent Dr Manish Mandal said institute director Dr NR Biswas held a meeting on Thursday morning before directing that the word virgin on the marital declaration form be immediately replaced with unmarried.', 12.873637136668265)
('The Indira Gandhi Institute of Medical Sciences (IGIMS) in Patna amended its marital declaration form on Thursday, replacing the word virgin with unmarried after controversy.', 8.8421443830275166)


In [382]:
print gold_summary


The Indira Gandhi Institute of Medical Sciences (IGIMS) in Patna on Thursday made corrections in its Marital Declaration Form by changing 'Virgin' option to 'Unmarried'. Earlier, Bihar Health Minister defined virgin as being an unmarried woman and did not consider the term objectionable. The institute, however, faced strong backlash for asking new recruits to declare their virginity in the form.


In [384]:
from nltk.stem.snowball import SnowballStemmer
from nltk import ngrams

n = 1

stemmer = SnowballStemmer("english")

gold_summary_slammed = ""
for word in gold_summary.split():
    gold_summary_slammed = gold_summary_slammed+stemmer.stem(word)+" "


machine_summary_slammed = ""
for word in machine_summary.split():
    machine_summary_slammed = machine_summary_slammed+stemmer.stem(word)+" "

n_gold = ngrams(gold_summary_slammed.split(" "),n)
n_machine = ngrams(machine_summary_slammed.split(" "),n)

gold_list = []

for gram in n_gold:
    gold_list.append(gram)

machine_list = []
for gram in n_machine:
    machine_list.append(gram)

print float(len(list(set(gold_list).intersection(machine_list))))/float(len(list(set(gold_list))))

0.384615384615
